In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd

Mounted at /content/gdrive


In [23]:
# קריאה של קובץ ה-CSV
df = pd.read_excel('/content/gdrive/MyDrive/time_series (1).xlsx')
df.head()

,timestamp,value
0,2025-06-28 12:00:52,18.5
1,2025-06-01 04:17:23,46.3
2,2025-06-10 17:02:57,76
3,2025-06-23 05:23:22,56.4
4,2025-06-05 07:20:08,67.9


In [3]:
df.shape

(1000000, 2)

In [4]:

# 1. בדיקת פורמט התאריך:
# המרת עמודת Timestamp לתאריך. אם המרה לא תצליח, התוצאה תהיה NaT (Not a Time).
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

# איתור שורות בהן ההמרה נכשלה (כלומר, פורמט לא תקין)
invalid_dates = df['timestamp'].isnull()
if invalid_dates.any():
    print("נמצאו תאריכים עם פורמט לא תקין בשורות הבאות:")
    print(df[invalid_dates])
else:
    print("כל פורמטי התאריכים תקינים.")


כל פורמטי התאריכים תקינים.


In [5]:

# 2. בדיקת כפילויות:
duplicates = df.duplicated()
if duplicates.any():
    print("נמצאו שורות כפולות:")
    print(df[duplicates])
else:
    print("לא נמצאו שורות כפולות.")


נמצאו שורות כפולות:
                 timestamp         value
16085  2025-06-12 22:31:23           NaN
26420  2025-06-05 01:39:21           NaN
39052  2025-06-10 09:54:06           NaN
40835  2025-06-05 12:07:30           NaN
49961  2025-06-03 16:20:32  not_a_number
...                    ...           ...
999300 2025-06-03 11:17:22           NaN
999429 2025-06-22 20:06:07           NaN
999478 2025-06-09 11:42:30           NaN
999508 2025-06-12 12:53:11           NaN
999536 2025-06-17 13:31:50  not_a_number

[2598 rows x 2 columns]


In [6]:
# 3. בדיקה נוספת: איתור ערכים חסרים (missing values)
missing_values = df.isnull().sum()
if missing_values.any():
    print("נמצאו ערכים חסרים בעמודות הבאות:")
    print(missing_values)
else:
    print("לא נמצאו ערכים חסרים.")


נמצאו ערכים חסרים בעמודות הבאות:
timestamp        0
value        99040
dtype: int64


In [7]:

# הסרת שורות כפולות מהדאטא
df = df.drop_duplicates()

# הסרת שורות שבהן יש ערכים חסרים (NaN)
df = df.dropna()

# הדפסה לבדיקת התוצאה
print(df.head())
df.shape

            timestamp value
0 2025-06-28 12:00:52  18.5
1 2025-06-01 04:17:23  46.3
2 2025-06-10 17:02:57    76
3 2025-06-23 05:23:22  56.4
4 2025-06-05 07:20:08  67.9


(900314, 2)

קוד להדפסת הממוצע עבור כל תאריך ושעה

In [24]:


# המרת עמודת Timestamp לפורמט תאריך ושעה
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

# המרת עמודת value לסוג מספרי
df['value'] = pd.to_numeric(df['value'], errors='coerce')

# יצירת עמודה 'start_time' עם זמן מעוגל לשעה
df['start_time'] = df['timestamp'].dt.floor('H')

# קיבוץ לפי start_time וחישוב ממוצע הערכים בעמודת value
grouped = df.groupby('start_time')['value'].mean().reset_index()

# שינוי שמות העמודות לפורמט הרצוי
grouped = grouped.rename(columns={'start_time': 'start time', 'value': 'avarage'})

print(grouped)


             start time    avarage
0   2025-06-01 00:00:00  50.562894
1   2025-06-01 01:00:00  49.939803
2   2025-06-01 02:00:00  49.457213
3   2025-06-01 03:00:00  50.181573
4   2025-06-01 04:00:00  48.611496
..                  ...        ...
715 2025-06-30 19:00:00  50.907897
716 2025-06-30 20:00:00  48.924813
717 2025-06-30 21:00:00  50.754128
718 2025-06-30 22:00:00  48.178125
719 2025-06-30 23:00:00  50.302817

[720 rows x 2 columns]


<ipython-input-24-dc45a0a9747e>:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['start_time'] = df['timestamp'].dt.floor('H')


In [9]:
import os


# המרת עמודת Timestamp לפורמט תאריך ושעה
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df['value'] = pd.to_numeric(df['value'], errors='coerce')

# יצירת עמודת Date המכילה רק את החלק התאריכי
df['Date'] = df['timestamp'].dt.date

# יצירת תיקייה לקבצים יומיים
daily_folder = "daily_files"
os.makedirs(daily_folder, exist_ok=True)

# רשימה לאיחוד תוצאות העיבוד לכל קובץ יומי
results = []

# 1. פיצול הקובץ לקבצים יומיים ועיבוד כל אחד מהם
for date in df['Date'].unique():
    daily_df = df[df['Date'] == date].copy()
    filename = os.path.join(daily_folder, f"{date}.csv")
    daily_df.to_csv(filename, index=False)

    daily_df['timestamp'] = pd.to_datetime(daily_df['timestamp'], errors='coerce')

    # יצירת עמודת start_time עם זמן מעוגל לשעה (למשל 2025-06-10 06:00:00)
    daily_df['start_time'] = daily_df['timestamp'].dt.floor('H')

    # קיבוץ לפי start_time וחישוב ממוצע הערכים בעמודת value
    daily_avg = daily_df.groupby('start_time')['value'].mean().reset_index()
    results.append(daily_avg)

# 2. איחוד כל תוצאות העיבוד לכל הקבצים היומיים ל-DataFrame אחד
final_df = pd.concat(results, ignore_index=True)


final_df = final_df.rename(columns={'start_time': 'start time', 'value': 'avarage'})

final_df.to_csv("final_hourly_averages.csv", index=False)



<ipython-input-9-55cf0130d71e>:27: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  daily_df['start_time'] = daily_df['timestamp'].dt.floor('H')
<ipython-input-9-55cf0130d71e>:27: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  daily_df['start_time'] = daily_df['timestamp'].dt.floor('H')
<ipython-input-9-55cf0130d71e>:27: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  daily_df['start_time'] = daily_df['timestamp'].dt.floor('H')
<ipython-input-9-55cf0130d71e>:27: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  daily_df['start_time'] = daily_df['timestamp'].dt.floor('H')
<ipython-input-9-55cf0130d71e>:27: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  daily_df['start_time'] = daily_df['timestamp'].dt.floor('H')
<ipython-input-9-55c

In [10]:
dff=pd.read_csv("final_hourly_averages.csv")
dff

,start time,avarage
0,2025-06-28 00:00:00,50.459136
1,2025-06-28 01:00:00,49.353554
2,2025-06-28 02:00:00,49.720972
3,2025-06-28 03:00:00,49.530114
4,2025-06-28 04:00:00,49.871658
...,...,...
715,2025-06-09 19:00:00,49.937367
716,2025-06-09 20:00:00,51.961717
717,2025-06-09 21:00:00,50.154417
718,2025-06-09 22:00:00,49.443729


תשובה לשאלה :3.	אם הנתונים מגיעים בזרימה (stream) במקום מקובץ, כיצד תתכנני את הפתרון כדי לעדכן את הממוצעים השעתיים בזמן אמת?  

נחזיק מבנה נתונים מסוג מילון שבו המפתח הוא השעה (0-23)   והערכים הם סכום הערכים ומספר הרשומות עבור אותה שעה.
כך בכל פעם שנכנסת רשומה חדשה אנחנו רק נצטרך לעדכן את המפתח המתאים במילון ובכל פעם שנרצה לדעת את הממוצע הנוכחי נוכל לחשב את זה מאד בקלות ע"י חלוקת הסכום המצטבר במספר הרשומות עבור כל שעה.

#שאלה 4

קריאת קובץ parquest שקיבלנו, וז לא אותו פורמט, לכן פשוט נמיר את הCSV ונעבוד עליו.

In [20]:
dfp=pd.read_parquet('/content/gdrive/MyDrive/time_series (4).parquet')
dfp

,timestamp,mean_value,median_value,std_dev,count
0,2025-06-01 00:00:00,49.985786,49.504345,15.794580,1402
1,2025-06-01 01:00:00,50.492752,51.083455,14.944562,1464
2,2025-06-01 02:00:00,49.627839,50.157670,14.466200,1461
3,2025-06-01 03:00:00,50.092934,49.914063,15.176829,1428
4,2025-06-01 04:00:00,49.514965,49.365432,14.852564,1481
...,...,...,...,...,...
691,2025-06-29 19:00:00,50.244835,50.248594,14.867060,1390
692,2025-06-29 20:00:00,49.963043,50.506529,15.218887,1433
693,2025-06-29 21:00:00,49.703735,50.177605,14.415797,1474
694,2025-06-29 22:00:00,49.816577,49.976943,15.531267,1408


In [11]:

# שם קובץ הפלט – קובץ Parquet
parquet_file = "output.parquet"

# שמירת הנתונים כקובץ Parquet
df.to_parquet(parquet_file, index=False)

print(f"נוצר קובץ Parquet: {parquet_file}")

נוצר קובץ Parquet: output.parquet


In [17]:

# קריאת קובץ Parquet
df2 = pd.read_parquet("output.parquet")

# המרת עמודת timestamp לפורמט תאריך ושעה
df2['timestamp'] = pd.to_datetime(df2['timestamp'], errors='coerce')

# המרת עמודת value לסוג מספרי
df2['value'] = pd.to_numeric(df['value'], errors='coerce')

# יצירת עמודת start_time עם זמן מעוגל לשעה (לדוגמה: 2025-06-10 06:00:00)
df2['start_time'] = df2['timestamp'].dt.floor('h')

# קיבוץ לפי start_time וחישוב ממוצע הערכים בעמודת value
result = df2.groupby('start_time')['value'].mean().reset_index()

# שינוי שמות העמודות לפורמט המבוקש
result = result.rename(columns={'start_time': 'start time', 'value': 'avarage'})

print(result)


             start time    avarage
0   2025-06-01 00:00:00  50.301709
1   2025-06-01 01:00:00  49.979784
2   2025-06-01 02:00:00  49.822960
3   2025-06-01 03:00:00  48.898772
4   2025-06-01 04:00:00  49.328026
..                  ...        ...
715 2025-06-30 19:00:00  50.448259
716 2025-06-30 20:00:00  49.546972
717 2025-06-30 21:00:00  49.438999
718 2025-06-30 22:00:00  49.061580
719 2025-06-30 23:00:00  48.964738

[720 rows x 2 columns]


מה שהיה צריך לשנות בקוד זה רק את צורת הקריאה, במקום

 read_excel-> read_parquest

 היתרונות באחסון המידע בפורמט זה:
 קובץ זה מאוחסן בעמודות , ולא בשורות כמו אקסל מה שמאפשר גישה מהירה ויעילה יותר לעמודות ספציפיות.
 הוא משמש לעבודה עם הרבה נתונים ו BIG DATA.